In [15]:
import pandas as pd
import os
import csv
import numpy as np
from matplotlib import pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [13]:
from Experimentos import Experimentos, labelLines

In [14]:
#experimentos sin FLIR: 11
#experimentos con FLIR: 22
for i, exp in enumerate(Experimentos):
   print(i, exp, (exp.flir is not None))

TypeError: 'Experimento' object is not subscriptable

In [ ]:
Experimentos[19].param.head()

In [3]:
#FUNCION Q DEVUELVE RANGO [1-7] SEGUN TEMP MAXIMA DEL EXP
def rango_temperatura(temp):
    if temp < 40:
        return 1
    if temp < 45:
        return 2
    if temp < 50:
        return 3
    if temp < 55:
        return 4
    if temp < 60:
        return 5
    if temp < 65:
        return 6
    if temp < 70:
        return 7

In [5]:
#LISTA para armar graficos de PERFIL agrupados por caudal
#es una lista donde en cada value hay un diccionario por experimento
listaporcaudal = []

for exp in Experimentos:
    holder = {} #es un diccionario 
    holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
    holder['espesor'] = exp.perfil['perfil: espesor']#toda la columna
    holder['largo'] = exp.perfil['perfil: distancia']#toda la columna
    holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
    holder['angulo'] = exp.param['angulo inclinacion'][0]
    holder["rango temperatura"] = rango_temperatura(exp.param["t max"][0])
    holder['uso'] = exp.param['uso manjar'][0]

    listaporcaudal.append(holder)

In [ ]:
#devuelve el valor que le pido recorriendo la lista creada arriba
#for i, exp in enumerate(listaporcaudal):
    #print(i,(exp['angulo']))

In [19]:
#LISTA para armar graficos de ANCHO VS ESPESOR agrupados por caudal
#es una lista donde en cada value hay un diccionario por experimento
listaporanchoyesp = []

for exp in Experimentos:
    holder = {} #es un diccionario 
    holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
    holder['espesor'] = exp.param['result: espesor max final']#toda la columna
    holder['ancho'] = exp.param['result: ancho max final']#toda la columna
    holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
    holder['angulo'] = exp.param['angulo inclinacion'][0]
    holder["rango temperatura"] = rango_temperatura(exp.param["t max"][0])

    listaporanchoyesp.append(holder)

In [23]:
for x in listaporanchoyesp:
    print(x['nombre'],x['ancho'].min())

5cc50C 0.245
1cc54C 0.24
25cc54C 0.255
10cc60C 0.242
25cc60C 0.245
1cc59C 0.25
1cc47C 0.27
1cc55C 0.26
25cc58C 0.295
25cc70C 0.3
10cc67C 0.275
10cc59C 0.276
5cc63C 0.266
5cc59C 0.278
10cc49C 0.287
25cc62C 0.295
1cc62C 0.288
5cc53C 0.295
5a25cc56C 0.28
5cc57C 0.28
25cc50C 0.272
10cc55C 0.295
1cc24C 0.29
10cc57C 0.295
10cc24C 0.295
5cc55C 0.29
5cc58C 0.286
1cc48C 0.265
1cc51C 0.285
5cc48C 0.29
10cc50C 0.305
1cc54,7C 0.252
1cc52C 0.245


In [8]:
#se arman GRUPOS SEGUN CAUDAL
from itertools import groupby
count = 0

#key es la lista donde estan los diccionarios de exp y DF
#group es que se agrupa segun el valor de 'caudal' en este caso

for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    count += 1
    #for exp in group:
        #muestra el nombre del grupo y los elementos que tiene
        #print(exp['nombre']) 

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [9]:
#EJES GRAFICOS PERFILES SEGUN CAUDAL
maximoejelargo = max([exp['largo'].max() for exp in listaporcaudal])
minimoejelargo = min([exp['largo'].min() for exp in listaporcaudal])
maximoejeespesor = max([exp['espesor'].max() for exp in listaporcaudal])
minimoejeespesor = min([exp['espesor'].min() for exp in listaporcaudal])

In [10]:
#EJES GRAFICOS ANCHOS MAXIMOS SEGUN CAUDAL
maximoejeancho = max([exp['ancho'].max() for exp in listaporanchoyesp])
minimoejeancho = min([exp['ancho'].min() for exp in listaporanchoyesp])
maximoejeespesor = max([exp['espesor'].max() for exp in listaporanchoyesp])
minimoejeespesor = min([exp['espesor'].min() for exp in listaporanchoyesp])

In [11]:
#GENERA PATCHES DE COLORES SEGUN TEMPERATURA
import matplotlib.patches as mpatches

pat1 = mpatches.Patch(color='C1', label='temp < 40')
pat2 = mpatches.Patch(color='C2', label='temp < 45')
pat3 = mpatches.Patch(color='C3', label='temp < 50')
pat4 = mpatches.Patch(color='C4', label='temp < 55')
pat5 = mpatches.Patch(color='C5', label='temp < 60')
pat6 = mpatches.Patch(color='C6', label='temp < 65')
pat7 = mpatches.Patch(color='C7', label='temp < 70')

numerocolores = 7
colors = ['pat{}'.format(i+1) for i in range(numerocolores)]

print(colors)

['pat1', 'pat2', 'pat3', 'pat4', 'pat5', 'pat6', 'pat7']


In [25]:
#CASI LISTO!!! FALTA LEYENDA LINEA PUNTEADA Y CONTINUA
#5 GRAFICOS DE PERFIL AGRUPADOS POR CAUDAL Y COLOREADOS SEGUN TEMEPRATURA
fig = plt.figure()
axes = [fig.add_subplot(3,2,i+1) for i in range(count)]
counter = 0

for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    ax = axes[counter]
    ax.set_xlim(minimoejelargo, maximoejelargo)
    ax.set_ylim(0, maximoejeespesor)
    ax.set_title('{}cc'.format(key*1000000))
    counter += 1
    
    for exp in group:
        #print("C{}".format(exp["rango temperatura"]))
        if exp['angulo'] == 15:
            ax.plot(exp['largo'], exp['espesor'], color='C{}'.format(exp['rango temperatura']), 
                    linestyle= '--', label=exp['nombre'])
        else:
            ax.plot(exp['largo'], exp['espesor'], color='C{}'.format(exp['rango temperatura']), label=exp['nombre'])
    
#    labelLines(ax.get_lines())
#linea1, = ax.plot([],[], label='15°', linestyle='--', color='black')
#linea2, = ax.plot([],[], label='10°', color='black')
#plt.legend([linea1, linea2], ['15°', '10°'])
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
plt.gca().add_artist(legend1)
fig.show()

1e-06


KeyError: 'ancho'

In [ ]:
Experimentos[25].perfil

In [ ]:
Experimentos[26].perfil

In [13]:
#1 GRAFICO DE PERFIL INLABEL NAME
for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    if key == 2.5e-05:
        c = 0
        for exp in group:
            #print("C{}".format(exp["rango temperatura"]))
            c = c + 1
            if exp['angulo'] == 15:
                plt.plot(exp['largo'], exp['espesor'], color='C{}'.format(exp['rango temperatura']), 
                        linestyle= '--', label=exp['nombre'])
            else:
                plt.plot(exp['largo'], exp['espesor'], color='C{}'.format(exp['rango temperatura']), label=exp['nombre'])
    
        plt.show()
        break
    
linea1 = ax.plot([],[], label='15°', linestyle='--', color='black')
linea2 = ax.plot([],[], label='10°', color='black')
plt.legend([linea1, linea2], ['15°', '10°'])
labelLines(plt.gca().get_lines(), xvals=np.linspace(0, 0.23, c+2)[1: -1])
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.115, 0.9), loc=1)
plt.legend(bbox_to_anchor=(1.115, 0.5), loc=1)
plt.gca().add_artist(legend1)
print(c)

1e-06
5e-06
1e-05
1.7e-05
2.5e-05
6


C:\Users\CCVV\Anaconda3b\lib\site-packages\matplotlib\legend.py:634: UserWarning: Legend does not support [<matplotlib.lines.Line2D object at 0x000002D598979278>] instances.
A proxy artist may be used instead.
See: http://matplotlib.org/users/legend_guide.html#using-proxy-artist
  "#using-proxy-artist".format(orig_handle)
C:\Users\CCVV\Anaconda3b\lib\site-packages\matplotlib\legend.py:634: UserWarning: Legend does not support [<matplotlib.lines.Line2D object at 0x000002D598AA1D68>] instances.
A proxy artist may be used instead.
See: http://matplotlib.org/users/legend_guide.html#using-proxy-artist
  "#using-proxy-artist".format(orig_handle)


In [112]:
for i, (key, group) in enumerate(groupby(sorted(listadinamica, key=lambda x: x['caudal']), key=lambda x: x['caudal'])):
    print(i, key)
    c = 0
    if key == 1e-05:
        for exp in group:
            c = c + 1
            if exp['angulo'] == 15:
                plt.loglog(exp['tiempo'], exp['distancia'], color='C{}'.format(exp['rango temperatura']),
                               linestyle='--', label = exp['nombre'])
            else:
                plt.loglog(exp['tiempo'], exp['distancia'], color='C{}'.format(exp['rango temperatura']), label = exp['nombre'])
        plt.show
        break

linea1= plt.plot([],[], label='15°', linestyle='--', color='black')
linea2= plt.plot([],[], label='10°', color='black')
plt.legend([linea1, linea2], ['15°', '10°'])
labelLines(plt.gca().get_lines(), xvals=np.linspace(-10, 95, c+2)[1: -1])
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1, 0.5), loc=1)
plt.legend(bbox_to_anchor=(1.11, 0.5), loc=1)
plt.gca().add_artist(legend1)
fig.show()

0 1e-06
1 5e-06
2 1e-05


C:\Users\CCVV\Anaconda3b\lib\site-packages\matplotlib\legend.py:634: UserWarning: Legend does not support [<matplotlib.lines.Line2D object at 0x000002D59A7454A8>] instances.
A proxy artist may be used instead.
See: http://matplotlib.org/users/legend_guide.html#using-proxy-artist
  "#using-proxy-artist".format(orig_handle)
C:\Users\CCVV\Anaconda3b\lib\site-packages\matplotlib\legend.py:634: UserWarning: Legend does not support [<matplotlib.lines.Line2D object at 0x000002D59CA36828>] instances.
A proxy artist may be used instead.
See: http://matplotlib.org/users/legend_guide.html#using-proxy-artist
  "#using-proxy-artist".format(orig_handle)


In [20]:
#LISTOOOO
#5 GRAFICOS DE PERFIL AGRUPADOS POR CAUDAL Y COLOREADOS SEGUN TEMEPRATURA 
#SOLO PRIMER USO DEL MANJAR
fig = plt.figure()
axes = [fig.add_subplot(3,2,i+1) for i in range(count)]
counter = 0

for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    ax = axes[counter]
    ax.set_xlim(minimoejelargo, maximoejelargo)
    ax.set_ylim(0, maximoejeespesor)
    ax.set_title('{}cc'.format(key*1000000))
    counter += 1
    linea1= ax.plot([],[], label='15°', linestyle='--', color='black')
    linea2= ax.plot([],[], label='10°', color='black')
    
    for exp in group:
        #print("C{}".format(exp["rango temperatura"]))
        if exp['uso'] == 1: 
            if exp['angulo'] == 15:
                ax.plot(exp['largo'], exp['espesor'], color='C{}'.format(exp['rango temperatura']), 
                        linestyle= '--')
            else:
                ax.plot(exp['largo'], exp['espesor'], color='C{}'.format(exp['rango temperatura']))
        
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
plt.gca().add_artist(legend1)
fig.show()

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [29]:
#LISTO!!!
#1 GRAFICO SCATTER DE ESPESOR VS ANCHO AGRUPADOS POR CAUDAL Y COLOREADOS X TEMPERATURA
fig, ax = plt.subplots()
counter = 0
marqer = ['o', 'v', '^', '<', '>']
plotmarqer = []

from matplotlib import markers

contadorkey = 0

for key, group in groupby(sorted(listaporanchoyesp, key=lambda x: x['caudal']), 
                          key=lambda x: x['caudal']):
    print(key)
    ax.set_xlim(minimoejeancho-0.02, maximoejeancho+0.02)
    ax.set_ylim(maximoejeespesor-0.02, maximoejeespesor+0.001)
    markadorkey = marqer[contadorkey]
    contadorkey += 1
    
    for i, exp in enumerate(group):
        if exp['angulo'] == 15:
            l = ax.plot(exp['ancho'], exp['espesor'], color='black', 
                           marker=markadorkey)
            print(exp['ancho'])
            #l.set_markeredgecolor('Black')
        else: 
            l = ax.plot(exp['ancho'], exp['espesor'], color='C{}'.format(exp['rango temperatura']), 
                           marker=markadorkey)
            print(exp['ancho'])
        if i == 0:
            plotmarqer.append(ax.scatter([], [], color='black', marker=markadorkey, label=exp['caudal']))
            print(exp['ancho'])


legendcolors2 = [pat1, pat2, pat3, pat4, pat5, pat6, pat7]

legend1 = plt.legend(handles=plotmarqer, loc= 3)
plt.legend(handles=legendcolors2, loc=2)
plt.gca().add_artist(legend1)
plt.show()

1e-06
0    0.24
Name: result: ancho max final, dtype: float64
0    0.24
Name: result: ancho max final, dtype: float64
0    0.25
Name: result: ancho max final, dtype: float64
0    0.27
Name: result: ancho max final, dtype: float64
0    0.26
Name: result: ancho max final, dtype: float64
0    0.288
Name: result: ancho max final, dtype: float64
0    0.29
Name: result: ancho max final, dtype: float64
0    0.265
Name: result: ancho max final, dtype: float64
0    0.285
Name: result: ancho max final, dtype: float64
0    0.252
Name: result: ancho max final, dtype: float64
0    0.245
Name: result: ancho max final, dtype: float64
5e-06
0    0.245
Name: result: ancho max final, dtype: float64
0    0.245
Name: result: ancho max final, dtype: float64
0    0.266
Name: result: ancho max final, dtype: float64
0    0.278
Name: result: ancho max final, dtype: float64
0    0.295
Name: result: ancho max final, dtype: float64
0    0.28
Name: result: ancho max final, dtype: float64
0    0.29
Name: result: an

In [25]:
#lista para armar figures de velocidad segun Q
listadinamica = []
#es una lista donde en cada value hay un diccionario por experimento
#ANOTAR EN UNA HOJA COMO SE VE Y USA ESTO PLS
for exp in Experimentos:
    holder = {} #es un diccionario 
    holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
    holder['tiempo'] = exp.dinamica['avance: tiempo']#toda la columna
    holder['distancia'] = exp.dinamica['avance: distancia desde punto eyeccion']#toda la columna
    holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
    holder['angulo'] = exp.param['angulo inclinacion'][0]
    holder["rango temperatura"] = rango_temperatura(exp.param["t max"][0])
        
    listadinamica.append(holder)

In [18]:
#FALTA: LEYENDA 25CC DENTRO DEL GRAFICO Y ARREGLAR LEYENDA TIPO LINEA Y ANGULO
#empezamo el PLOT LINEA DINAMICAS SEGUN CAUDAL
fig = plt.figure()
axes = []

#este plot hace cuantos subplot necesite segun un contador que da el numero de
#grupos generados en el groupby
for x in range(count):
    axes.append(fig.add_subplot(2, 3, x + 1))

#UNA COLUMNA DE GRAFICOS CON VARIAS FILAS SEGUN GRUPO
#se generan los ejes de cada plot
#i es el indice de lista y luego cada value en lista (???)
for i, (key, group) in enumerate(groupby(sorted(listadinamica, key=lambda x: x['caudal']), key=lambda x: x['caudal'])):
    print(i, key)

    for exp in group:
        axes[i].legend([key])
        if exp['angulo'] == 15:
            axes[i].loglog(exp['tiempo'], exp['distancia'], color='C{}'.format(exp['rango temperatura']),
                           linestyle='--')
        else:
            axes[i].loglog(exp['tiempo'], exp['distancia'], color='C{}'.format(exp['rango temperatura']))
        axes[i].legend([key])
            
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.gca().add_artist(legend1)

linea1= ax.plot([],[], label='15°', linestyle='--', color='black')
linea2= ax.plot([],[], label='10°', color='black')       
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
labelLines(plt.gca().get_lines(),align=False,color='k')
fig.show()

0 1e-06
1 5e-06
2 1e-05
3 1.7e-05
4 2.5e-05
x label location is outside data range!
x label location is outside data range!
x label location is outside data range!


In [29]:
#un plot de dinamica con marcador
plt.scatter(Experimentos[6].dinamica['avance: tiempo'], Experimentos[6].dinamica['avance: largo total flujo'], marker = 'x')

In [32]:
exp = Experimentos[30]
exp

10cc50C.csv

In [33]:
exp.dinamica.tail()

,Unnamed: 0,avance: tiempo,avance: distancia desde punto eyeccion,avance: distancia hacia atrás,avance: largo total flujo,avance: ancho maximo,volumen_emitido,anchoporlargo,area_avance,espesor
44,44,5441.333333,0.360,0.074,0.434,0.305,0.001493,0.033092,0.103910,0.021546
45,45,5967.333333,0.362,0.074,0.436,0.305,0.001493,0.033245,0.104389,0.021447
46,46,9159.333333,0.368,0.074,0.442,0.305,0.001493,0.033702,0.105826,0.021156
47,47,9986.333333,0.368,0.074,0.442,0.305,0.001493,0.033702,0.105826,0.021156
48,48,76919.333333,0.368,0.074,0.442,0.305,0.001493,0.033702,0.105826,0.021156


In [30]:
#(exp.dinamica['avance: ancho maximo']/2)*(exp.dinamica['avance: largo total flujo']/2)

In [31]:
rho_manjar = 1340
kilos_manjar = 2
mitad = 2
pi = 3.14

for exp in Experimentos:
    exp.dinamica['volumen_emitido'] = exp.param['caudal'][0]*exp.dinamica['avance: tiempo']
    exp.dinamica['anchoporlargo'] = (exp.dinamica['avance: ancho maximo']/mitad)*(exp.dinamica['avance: largo total flujo']/mitad)
    exp.dinamica['area_avance'] = exp.dinamica['anchoporlargo']*pi
    volumen_total = kilos_manjar/rho_manjar
    exp.dinamica['volumen_emitido'][exp.dinamica['volumen_emitido'] > volumen_total] = volumen_total
    exp.dinamica['espesor'] = 3*exp.dinamica['volumen_emitido']/(2*exp.dinamica['area_avance'])

C:\Users\CCVV\Anaconda3b\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [34]:
exp.dinamica.describe()

,Unnamed: 0,avance: tiempo,avance: distancia desde punto eyeccion,avance: distancia hacia atrás,avance: largo total flujo,avance: ancho maximo,volumen_emitido,anchoporlargo,area_avance,espesor
count,49.00000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,48.000000
mean,24.00000,3020.438095,0.221551,0.063000,0.284551,0.231000,0.001041,0.019332,0.060702,0.028525
std,14.28869,11035.305590,0.119660,0.017264,0.135020,0.089953,0.000573,0.012213,0.038349,0.005500
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021156
25%,12.00000,45.666667,0.120000,0.055000,0.175000,0.160000,0.000457,0.007000,0.021980,0.023033
50%,24.00000,182.000000,0.240000,0.074000,0.314000,0.284000,0.001493,0.022294,0.070003,0.030423
75%,36.00000,1969.333333,0.330000,0.074000,0.404000,0.305000,0.001493,0.030805,0.096728,0.032725
max,48.00000,76919.333333,0.368000,0.074000,0.442000,0.305000,0.001493,0.033702,0.105826,0.041904


In [35]:
#lista para armar figures de velocidad segun Q
listaespesor = []
#es una lista donde en cada value hay un diccionario por experimento
#ANOTAR EN UNA HOJA COMO SE VE Y USA ESTO PLS
for exp in Experimentos:
    holder = {} #es un diccionario 
    holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
    holder['tiempo'] = exp.dinamica['avance: tiempo']#toda la columna
    holder['espesor'] = exp.dinamica['espesor']#toda la columna
    holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
    holder['angulo'] = exp.param['angulo inclinacion'][0]
    holder["rango temperatura"] = rango_temperatura(exp.param["t max"][0])
        
    listaespesor.append(holder)

In [36]:
fig= plt.plot(Experimentos[30].dinamica['avance: tiempo'], Experimentos[30].dinamica['espesor'], marker = 'x')

In [38]:
#Graficos: ESPESOR vs Tiempo

fig = plt.figure()
axes = []

for x in range(count):
    axes.append(fig.add_subplot(2, 3, x + 1))

for i, (key, group) in enumerate(groupby(sorted(listaespesor, key=lambda x: x['caudal']), key=lambda x: x['caudal'])):
    print(i, key)

    for exp in group:
        axes[i].legend([key])
        if exp['angulo'] == 15:
            axes[i].scatter(exp['tiempo'], exp['espesor'], color='C{}'.format(exp['rango temperatura']),
                           linestyle='--')
        else:
            axes[i].scatter(exp['tiempo'], exp['espesor'], color='C{}'.format(exp['rango temperatura']))
        axes[i].legend([key])
            
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.gca().add_artist(legend1)

linea1= ax.plot([],[], label='15°', linestyle='--', color='black')
linea2= ax.plot([],[], label='10°', color='black')       
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
fig.show()

0 1e-06
1 5e-06
2 1e-05
3 1.7e-05
4 2.5e-05


In [39]:
#lista para armar figures de velocidad segun Q
listaancho = []
#es una lista donde en cada value hay un diccionario por experimento
#ANOTAR EN UNA HOJA COMO SE VE Y USA ESTO PLS
for exp in Experimentos:
    holder = {} #es un diccionario 
    holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
    holder['tiempo'] = exp.dinamica['avance: tiempo']#toda la columna
    holder['ancho'] = exp.dinamica['avance: ancho maximo']#toda la columna
    holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
    holder['angulo'] = exp.param['angulo inclinacion'][0]
    holder["rango temperatura"] = rango_temperatura(exp.param["t max"][0])
        
    listaancho.append(holder)

In [41]:
#Graficos: ANCHO VS Tiempo
fig = plt.figure()
axes = []

for x in range(count):
    axes.append(fig.add_subplot(2, 3, x + 1))

for i, (key, group) in enumerate(groupby(sorted(listaancho, key=lambda x: x['caudal']), key=lambda x: x['caudal'])):
    print(i, key)

    for exp in group:
        axes[i].legend([key])
        if exp['angulo'] == 15:
            axes[i].loglog(exp['tiempo'], exp['ancho'], color='C{}'.format(exp['rango temperatura']),
                           linestyle='--')
        else:
            axes[i].loglog(exp['tiempo'], exp['ancho'], color='C{}'.format(exp['rango temperatura']))
        axes[i].legend([key])
            
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.gca().add_artist(legend1)

linea1= ax.plot([],[], label='15°', linestyle='--', color='black')
linea2= ax.plot([],[], label='10°', color='black')       
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
fig.show()

0 1e-06
1 5e-06
2 1e-05
3 1.7e-05
4 2.5e-05
